### Production Features Pipeline

In [ ]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
# WEBSCRAPER = "SELENIUM"

In [ ]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  # for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir("..")


from src.utils.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data.cleaning import (
    process_games,
    add_TARGET,
)

from src.data.build_features import (
    process_features,
)

from src.utils.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

from src.utils.constants import (
    FEATURE_GROUP_VERSION,
)

DATAPATH = Path(r"data")

**Load API keys**

In [ ]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]
except:
    raise Exception("Set environment variable HOPSWORKS_API_KEY")

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == "SCRAPINGANT":
    try:
        SCRAPINGANT_API_KEY = os.environ["SCRAPINGANT_API_KEY"]
    except:
        raise Exception("Set environment variable SCRAPINGANT_API_KEY")
    driver = None

elif WEBSCRAPER == "SELENIUM":
    driver = activate_web_driver()
    SCRAPINGANT_API_KEY = ""

**Scrape New Completed Games and Format Them**

In [ ]:
df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print("No new games to process")
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new["SEASON"].max()

    print(SEASON)
    print(df_new.head())